![title](./pics/dd_logo.png) 


# Getting started

**deep**doctection is a package that can be used to extract text from complex structured documents. It also allows you to run vision/text and multi-modal models in an end-to-end pipeline. Inputs can be native PDFs or images. It is very versatile.

Compared to most other parsers, **deep**doctection offers extensive configurability. We will explore these capabilities in more detail in other notebooks.

This notebook will introduce you to the essential basics of document parsing with **deep**doctection.

We assume that, in addition to **deep**doctection, the transformers and python-doctr packages are installed if you are using PyTorch as your deep learning framework. 

If you are using TensorFlow, tensorpack must be installed instead.

Please note that TensorFlow is no longer supported from Python 3.11 onward, and its functionality within **deep**doctection is significantly more limited.

We recommend not using the TensorFlow setup anymore.

You will also need matplotlib that you can install with 

```
pip install matplotlib
```

In [1]:
from pathlib import Path
from matplotlib import pyplot as plt
from IPython.core.display import HTML

import deepdoctection as dd

/Users/janismeyer/Projekte/deepdoctection/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[0523 22:14.15 @file_utils.py:31]  INF  PyTorch version 2.2.2 available.
[0523 22:14.15 @file_utils.py:69]  INF  Disabling Tensorflow because USE_TORCH is set


## Sample

Take an image (e.g. .png, .jpg, ...). If you take the example below you'll maybe need to change ```image_path```.

In [ ]:
image_path = Path.cwd() / "pics/samples/sample_2/sample_2.png"

# viz_handler is a helper class that helps you e.g. with reading or writing images
image = dd.viz_handler.read_image(image_path)
plt.figure(figsize = (25,17))
plt.axis('off')
plt.imshow(image)

![title](./pics/samples/sample_2/sample_2.png)

## Analyzer

Next, we instantiate the **deep**doctection analyzer. The analyzer is an example of a pipeline that can be built depending on the problem you want to tackle. This particular pipeline is built from various building blocks. We will come back to this later. 

We will be using the default configuration.

In [ ]:
analyzer = dd.get_dd_analyzer()

## Analyze methods

Once all models have been loaded, we can process a directory with images (.png, .jpf) or multi page PDF-documents. You can either set `path='path/to/dir'` if you have a folder of images or `path='path/to/my/doc.pdf'` if you have a pdf document. 

You will receive an error if your path points to a single image. Processing images requires to pass the path to the base image directory.

In [ ]:
path = Path.cwd() / "pics/samples/sample_2"

df = analyzer.analyze(path=path)

With

```
df = analyzer.analyze(path=path)
````

nothing has actually been processed yet.

The method ```analyzer.analyze(path=path)``` does not (yet) return a JSON object, but rather a specialized subclass of the ```DataFlow``` class. Essentially, it behaves like a [generator](https://wiki.python.org/moin/Generators). 

Before starting the iteration, we must call:

In [ ]:
df.reset_state() 

Now you can traverse through all the values of the `Dataflow` simply by using a `for`-loop or the `next` function. Let's go!  

In [6]:
doc=iter(df)
page = next(doc)

[0523 22:14.35 @doctectionpipe.py:103]  INF  Processing sample_2.png
Unused or unrecognized kwargs: masks_path, annotations.
[0523 22:14.37 @context.py:133]  INF  ImageLayoutService total: 2.3095 sec.
[0523 22:14.37 @context.py:133]  INF  AnnotationNmsService total: 0.002 sec.
Unused or unrecognized kwargs: masks_path, annotations.
[0523 22:14.38 @context.py:133]  INF  SubImageLayoutService total: 0.3684 sec.
[0523 22:14.38 @context.py:133]  INF  PubtablesSegmentationService total: 0.0066 sec.
[0523 22:14.38 @context.py:133]  INF  ImageLayoutService total: 0.4052 sec.
[0523 22:14.39 @context.py:133]  INF  TextExtractionService total: 0.9374 sec.
[0523 22:14.39 @context.py:133]  INF  MatchingService total: 0.0059 sec.
[0523 22:14.39 @context.py:133]  INF  TextOrderService total: 0.0301 sec.


## Page

Let's see what we got back. For each iteration we receive a `Page` object. This object stores all informations that have been collected from a page document when running through the pipeline. 

In [7]:
type(page)

deepdoctection.datapoint.view.Page

Let's also have a look on some top level information. 

In [8]:
print(f" height: {page.height} \n width: {page.width} \n file_name: {page.file_name} \n document_id: {page.document_id} \n image_id: {page.image_id}\n")

 height: 2339 
 width: 1654 
 file_name: sample_2.png 
 document_id: c1776412-857f-3102-af7c-1869139a278d 
 image_id: c1776412-857f-3102-af7c-1869139a278d



`document_id` and `image_id` are the same. The reason is because we only process a single image. The naming convention silently assumes that we deal with a one page document. Once we process multi page PDFs `document_id` and `image_id` differ.

With `get_attribute_names()` you get a list of all attributes. 

In [9]:
page.get_attribute_names()

{'angle',
 'chunks',
 'document_id',
 'document_summary',
 'document_type',
 'figures',
 'file_name',
 'language',
 'layouts',
 'location',
 'pad_bottom',
 'pad_left',
 'pad_right',
 'pad_top',
 'page_number',
 'residual_layouts',
 'size',
 'tables',
 'text',
 'words'}

In [10]:
page.document_type, page.language

(None, None)

`page.document_type` and `page.language` both return None. The reason is that the analyzer has no component for predicting a document type or a language. If you want that, you need to build a custom pipeline. Check this [notebook](Using_LayoutLM_for_sequence_classification.ipynb) for further information.

## Layout segments

We can visualize detected layout segments. If you set `interactive=True` a viewer will pop up. Use `+` and `-` to zoom out/in. Use `q` to close the page.

Alternatively, you can visualize the output with matplotlib.

In [ ]:
image = page.viz()
plt.figure(figsize = (25,17))
plt.axis('off')
plt.imshow(image)

![title](./pics/output_16_1.png)

Let's have a look at other attributes. We can use the `text` property to get the content of the document. You will notice that the table is not included. You can therefore filter tables from the other content. In fact you can even filter on every layout segment.

In [12]:
print(page.text)

Die W-Pools der DWS Gruppe werden einer angemessenen Anpassung der Risiken unterzogen, die die Adjustierung ex ante als auch ex post umfasst. Die angewandte robuste Method d soll sicherstellen, dass bei der Festlegung der W sowohl der risikoadjustierten Leistung als auch der Kapital- und Liguiditatsaustattung der DWS Gruppe Rechnung getragen wird. Die mittlung des Gesamtbetrags der W orientiert sich primàr an 0 der Tragfâhigkeit fur die DWS Gruppe (das heiBt, was kann" die DWS Gruppe langfristig an) Wi im Einklang mit regulatorisch Anforderungen gewâhren) und ( der Leistung (das heilt, was sollte" die DWS Gruppe an W gewâhren, um fur eine angemessene leistungsbezogene Vergutung zu sorgen und gleichzeit den langfristigen Erfolg des Unternehmens zu sichern).
Die DWS Gruppe hat fûr die Festlegung der W auf Ebene der individuellen Mitarbeiter die Grundsâtze for die Festlegung der variablen Vergutung" eingefuhrt. Diese enthalten Informatione Uber die Faktoren und MessgroBen, die bei Entsche

You can get the individual layout segments like `text`, `title`, `list` or `figure`. Layout segments also have various attributes. 

In [27]:
for layout in page.layouts:
    print("--------------")
    print(f"Layout segment: {layout.category_name}, score: {layout.score}, reading_order: {layout.reading_order}, bounding_box: {layout.bounding_box},\n annotation_id: {layout.annotation_id} \n \ntext: {layout.text} \n \n")

--------------
Layout segment: text, score: 0.9416185021400452, reading_order: 5, bounding_box: Bounding Box(absolute_coords: True,ulx: 137, uly: 768, lrx: 1518, lry: 825),
 annotation_id: 4dba19ad-12d7-346d-902c-aff8c602d724 
 
text: Nach der hervorragenden Entwicklung im Jahr 2017 hatte die globale easealpibande 2018 mit einigen Schwierigkeiten zu kâmpfen. Grûnde waren unguinstige Marktbedin- gungen, stârkere geopolitische Spannungen und die negative Stimmung unter den Anlegern, vor allem am europàischen Retail-Markt. Auch die DWS Gruppe blieb von dieser Entwicklung nicht verschont. 
 

--------------
Layout segment: text, score: 0.9415092468261719, reading_order: 9, bounding_box: Bounding Box(absolute_coords: True,ulx: 138, uly: 1048, lrx: 1518, lry: 1206),
 annotation_id: b6addb88-ca5a-3c04-afc5-35dfba05712c 
 
text: GemaBs Gesetz vom 17 Dezember 2010 Uber die Organismen fûr gemeinsame Anlagen (in seiner jeweils gultigen Fassung) sowie den ESMA-Leitlinien unter Berlcksichtigung der

You can also get the layout segments from the `chunks` attribute. The output is a list of tuples with the most essential meta data for each layout segment, namely: `document_id, image_id, page_number, annotation_id, reading_order, category_name` and `text`.

In [14]:
page.chunks[0]

('c1776412-857f-3102-af7c-1869139a278d',
 'c1776412-857f-3102-af7c-1869139a278d',
 0,
 'e967096b-8c4a-3e3e-99dd-99b02ea0bff4',
 1,
 <LayoutType.TEXT>,
 'Die W-Pools der DWS Gruppe werden einer angemessenen Anpassung der Risiken unterzogen, die die Adjustierung ex ante als auch ex post umfasst. Die angewandte robuste Method d soll sicherstellen, dass bei der Festlegung der W sowohl der risikoadjustierten Leistung als auch der Kapital- und Liguiditatsaustattung der DWS Gruppe Rechnung getragen wird. Die mittlung des Gesamtbetrags der W orientiert sich primàr an 0 der Tragfâhigkeit fur die DWS Gruppe (das heiBt, was kann" die DWS Gruppe langfristig an) Wi im Einklang mit regulatorisch Anforderungen gewâhren) und ( der Leistung (das heilt, was sollte" die DWS Gruppe an W gewâhren, um fur eine angemessene leistungsbezogene Vergutung zu sorgen und gleichzeit den langfristigen Erfolg des Unternehmens zu sichern).')

Tables cannot be retrieved from `page.layouts`. They have a special `page.tables` which is a python list of table objects. In our situation, only one table has been detected. 

In [15]:
len(page.tables)

1

Let's have a closer look at the table. 

In [16]:
table = page.tables[0]
table.get_attribute_names()

{'bbox',
 'cells',
 'columns',
 'csv',
 'html',
 'item',
 'layout_link',
 'max_col_span',
 'max_row_span',
 'np_image',
 'number_of_columns',
 'number_of_rows',
 'reading_order',
 'rows',
 'text',
 'words'}

In [30]:
print(f" number of rows: {table.number_of_rows} \n number of columns: {table.number_of_columns} \n reading order: {table.reading_order}, \n score: {table.score}")

 number of rows: 8 
 number of columns: 2 
 reading order: None, 
 score: 0.8250970840454102


There is no reading order. The reason is that we have excluded tables from having a specific reading order position because we want to separate tables from the narrative text. Only layout segments with a `reading_order` not equal to `None`  will be added to the `page.text` string. 

This is pure customizing and we can change the customizing so that tables are part of the narrative text. We will come to this in another tutorial when talking about customization.

You can get an html, csv or text version of the table.  

In [18]:
HTML(table.html)

Jahresdurchschnitt der Mitarbeiterzahl,139
Gesamtvergutung?,EUR 15.315. .952
Fixe Vergutung,EUR 13.151.856
Variable Vergutung,EUR 2.164.096
davon: Carried Interest,EURO
Gesamtvergutung fur Senior Management,EUR 1.468.434
Gesamtvergutung fûr sonstige Risikotrâger,EUR 324.229
Gesamtvergutung fur Mitarbeiter mit Kontrollfunktionen,EUR 554.046


Use `table.csv` to load the table into a Pandas Dataframe.

In [19]:
table.csv  #pd.DataFrame(table.csv, columns=["Key", "Value"])

[['Jahresdurchschnitt der Mitarbeiterzahl ', '139 '],
 ['Gesamtvergutung? ', 'EUR 15.315. .952 '],
 ['Fixe Vergutung ', 'EUR 13.151.856 '],
 ['Variable Vergutung ', 'EUR 2.164.096 '],
 ['davon: Carried Interest ', 'EURO '],
 ['Gesamtvergutung fur Senior Management ', 'EUR 1.468.434 '],
 ['Gesamtvergutung fûr sonstige Risikotrâger ', 'EUR 324.229 '],
 ['Gesamtvergutung fur Mitarbeiter mit Kontrollfunktionen ', 'EUR 554.046 ']]

There is also a string representation of the table.

In [20]:
table.text

'Jahresdurchschnitt der Mitarbeiterzahl  139  \n Gesamtvergutung?  EUR 15.315. .952  \n Fixe Vergutung  EUR 13.151.856  \n Variable Vergutung  EUR 2.164.096  \n davon: Carried Interest  EURO  \n Gesamtvergutung fur Senior Management  EUR 1.468.434  \n Gesamtvergutung fûr sonstige Risikotrâger  EUR 324.229  \n Gesamtvergutung fur Mitarbeiter mit Kontrollfunktionen  EUR 554.046  \n'

The method `kv_header_rows(row_number)` allows returning column headers and cell contents as key-value pairs for entire rows. Admittedly, the example here is flawed because the table lacks column headers. In fact, the table recognition model determines whether and where a column has a header. In this case, the prediction was incorrect.

However, the principle becomes clear: we receive a dictionary with the schema 

```{(column_number, column_header(column_number)): cell(row_number, column_number).text}```.

In [21]:
table.kv_header_rows(2)

{(1, 'Jahresdurchschnitt der Mitarbeiterzahl'): 'Gesamtvergutung?',
 (2, '139'): 'EUR 15.315. .952'}

Let's go deeper down the rabbit hole. A `Table` has cells and we can even get the text of one particular cell. Note that the output list is not sorted by row or column. But you can quickly sort the output according to your preferences.

In [22]:
cell = table.cells[0]
cell.get_attribute_names()

{'bbox',
 'body',
 'column_header',
 'column_number',
 'column_span',
 'header',
 'layout_link',
 'np_image',
 'projected_row_header',
 'reading_order',
 'row_header',
 'row_number',
 'row_span',
 'spanning',
 'text',
 'words'}

In [23]:
print(f"column number: {cell.column_number} \n row_number: {cell.row_number}  \n bounding_box: {cell.bounding_box} \n text: {cell.text} \n annotation_id: {cell.annotation_id}")

column number: 1 
 row_number: 8  
 bounding_box: Bounding Box(absolute_coords: True,ulx: 1, uly: 183, lrx: 640, lry: 210) 
 text: Gesamtvergutung fur Mitarbeiter mit Kontrollfunktionen 
 annotation_id: ad4eba10-411c-357f-941e-8084685e8bf8


Still not down yet, we have a list of `Word`s.  

In [32]:
for word in cell.words:
    print(f"score: {word.score} \n characters: {word.characters} \n reading_order: {word.reading_order} \n bounding_box: {word.bounding_box}")

score: 0.6492854952812195 
 characters: Kontrollfunktionen 
 reading_order: 5 
 bounding_box: Bounding Box(absolute_coords: False,ulx: 0.25488281, uly: 0.63085938, lrx: 0.33984375, lry: 0.64160156)
score: 0.6608988046646118 
 characters: Mitarbeiter 
 reading_order: 3 
 bounding_box: Bounding Box(absolute_coords: False,ulx: 0.18457031, uly: 0.63085938, lrx: 0.23632812, lry: 0.64160156)
score: 0.5716732740402222 
 characters: Gesamtvergutung 
 reading_order: 1 
 bounding_box: Bounding Box(absolute_coords: False,ulx: 0.08398438, uly: 0.62988281, lrx: 0.17089844, lry: 0.64355469)
score: 0.5945571064949036 
 characters: mit 
 reading_order: 4 
 bounding_box: Bounding Box(absolute_coords: False,ulx: 0.23535156, uly: 0.62988281, lrx: 0.25585938, lry: 0.64257812)
score: 0.6204714775085449 
 characters: fur 
 reading_order: 2 
 bounding_box: Bounding Box(absolute_coords: False,ulx: 0.16796875, uly: 0.62988281, lrx: 0.18652344, lry: 0.64257812)


You can see that `Word`s have a `reading_order`, which refers only to the order of the words contained within the `cell`. The `reading_order` applies on two levels: to layout segments within a page and to words within a layout segment.

There are additional reserved attributes, but most of them are not determined by this pipeline.

In [25]:
word.get_attribute_names()

{'bbox',
 'block',
 'character_type',
 'characters',
 'handwritten',
 'layout_link',
 'np_image',
 'printed',
 'reading_order',
 'tag',
 'text_line',
 'token_class',
 'token_tag'}

## Saving and reading

You can use the `save` method to save the result of the analyzer in a `.json` file. Setting `image_to_json=True` you will also save image as b64 encoding in the file. Beware, the files can be quite large then. 

In [ ]:
page.save(image_to_json=True, path=Path.cwd() / "pics/samples/sample_2/sample_2.json")

Having saved the results you can easily parse the file into the `Page` format without loosing any information. The `page` instance below has almost the same structure as the `page` instance returned from the `analyzer` with only some lower level data structure missing that can be reconstructed, though.

In [33]:
page = dd.Page.from_file(file_path=Path.cwd() / "pics/samples/sample_2/sample_2.json")

## Where to go from here

There are several options: 

- You can check this [**notebook**](./Analyzer_More_On_Parsing.ipynb) where we process a different page and explore some more features about the parsed results.

- Maybe you want to switch between different models or want to learn, how you can configure the analyzer. Then the
  [**configuration notebook**](./Analyzer_Configuration.ipynb) might be interesting.

- If you want to get a deeper understanding how a pipeline is composed, we suggest to look at the [**pipeline notebook**](./Pipelines.ipynb).